In [1]:
# Necessary imports
import pandas as pd
import numpy as np
import math

In [2]:
# Load data from excel file
stock_transfers_aggregated = pd.read_excel('../data/stock_transfers_aggregated.xlsx')
sales_aggregated = pd.read_excel('../data/sales_aggregated.xlsx')

In [3]:
# Replace whitespace of column titles with underscores
stock_transfers_aggregated.columns = [title.replace(' ', '_') for title in stock_transfers_aggregated.columns]
sales_aggregated.columns = [title.replace(' ', '_') for title in sales_aggregated.columns]

In [4]:
stock_transfers_aggregated

,Simulation_Round,Simulation_Step,Storage_Location,Quantity
0,4,20,02N,14086
1,4,20,02S,10000
2,4,20,02W,16342
3,4,19,02S,9260
4,4,18,02N,16089
...,...,...,...,...
191,1,8,02S,5000
192,1,8,02W,5000
193,1,5,02N,2500
194,1,5,02S,2500


In [5]:
sales_aggregated

,Material_Description,Material_Number,Quantity,Cost,Net_Value
0,500g Nut Muesli,CC-F01,179816,161834.40,929827.70
1,500g Blueberry Muesli,CC-F02,233000,286590.00,968148.96
2,500g Raisin Muesli,CC-F04,158083,124885.57,768773.76
3,1kg Nut Muesli,CC-F11,171283,277478.46,816037.41
4,1kg Blueberry Muesli,CC-F12,184644,419141.88,985521.49
5,1kg Strawberry Muesli,CC-F13,155000,353400.00,821105.60


In [6]:
# Calculate margin/pc of each product
sales_aggregated = sales_aggregated.assign(Margin=lambda x: (x.Net_Value - x.Cost) / x.Quantity)

In [7]:
# Calculate average margin of all products
margin_sum = 0

for index, row in sales_aggregated.iterrows():
    margin_sum += row.Margin
    
average_margin = round(margin_sum / sales_aggregated.shape[0], 2)

In [8]:
average_margin

3.42

In [9]:
# Shipping cost per shipping == 1 000€
# Calculate the shipping quantity to cover the costs of shipping
coverage_quantity = int(math.ceil(1000 / average_margin))

In [10]:
coverage_quantity

293

In [11]:
# Check which stock transfers lost us money
non_coverage_transfers = stock_transfers_aggregated[stock_transfers_aggregated.Quantity <= coverage_quantity].sort_values(by='Quantity')
non_coverage_transfers

,Simulation_Round,Simulation_Step,Storage_Location,Quantity
102,3,4,02N,1


In [96]:
groupby = stock_transfers_aggregated
groupby['Round_Step'] = groupby['Simulation_Round'].astype(str) + groupby['Simulation_Step'].astype(str)
groupby = groupby.drop(columns=['Simulation_Round', 'Simulation_Step'])
groupby = groupby.groupby(['Round_Step', 'Storage_Location'], as_index=False).sum()

In [97]:
groupby

,Round_Step,Storage_Location,Quantity
0,111,02N,4750
1,111,02S,5000
2,111,02W,5250
3,114,02N,17750
4,114,02S,20000
...,...,...,...
191,48,02S,4728
192,48,02W,5235
193,49,02N,996
194,49,02S,5754


In [108]:
groupby_location = groupby.groupby(['Round_Step'], as_index=False).count()
groupby_location = groupby_location.drop(columns=['Quantity'], axis=1)

In [109]:
groupby_location

,Round_Step,Storage_Location
0,111,3
1,114,3
2,116,3
3,117,1
4,118,3
...,...,...
64,45,3
65,46,3
66,47,3
67,48,3


In [110]:
groupby_quantity = groupby.groupby(['Round_Step'], as_index=False).sum()

In [111]:
groupby_quantity

,Round_Step,Quantity
0,111,15000
1,114,52500
2,116,68892
3,117,567
4,118,68297
...,...,...
64,45,27594
65,46,26266
66,47,14990
67,48,13036


In [113]:
merged = pd.merge(groupby_location, groupby_quantity, on='Round_Step')

In [114]:
merged

,Round_Step,Storage_Location,Quantity
0,111,3,15000
1,114,3,52500
2,116,3,68892
3,117,1,567
4,118,3,68297
...,...,...,...
64,45,3,27594
65,46,3,26266
66,47,3,14990
67,48,3,13036


In [115]:
non_coverage_transfers_fixcosts = merged[merged.Quantity <= (merged['Storage_Location'] * 293) + 7091].sort_values(by='Quantity')
non_coverage_transfers_fixcosts

,Round_Step,Storage_Location,Quantity
3,117,1,567
41,320,3,1342
15,215,2,1382
51,411,2,1990
5,119,1,3760
53,413,1,4345
7,15,3,7500
55,415,3,7887
